In [27]:
import numpy as np
import sympy as sy
import scipy.linalg as la
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.auto import tqdm

# Volume 1: Conditioning and Stability.
    Ethan Crawford
    Math 347
    2/6/23

Lab Objective: The condition number of a function measures how sensitive that function is to changes in the input. On the other hand, the stability of an algorithm measures how accurately that algorithm computes the value of a function from exact input. Both of these concepts are important for answering the crucial question, “is my computer telling the truth?” In this lab, I examine the conditioning of common linear algebra problems, including computing polynomial roots and matrix eigenvalues. I also present an example to demonstrate how two different algorithms for the same problem may not have the same level of stability.

<hr>

## Conditioning

The *absolute condition number* is a measure of how sensitive a function is to small perturbations in its input. Specifically, it is defined as the maximum ratio of the relative change in the function's output to the relative change in its input, over all possible perturbations.

Mathematically, the *absolute condition number* of a function $f : \R^m \to \R^n$ at a point $x \in \R^m$ is defined by 

$$ \hat{\kappa}(x) = \lim_{\delta \to 0^{+}} \sup_{\|h\| < \delta} \frac{\|f(x+h) - f(x)\|}{\|h\|} $$

The absolute condition number measures the worst-case amplification of relative errors in the input to relative errors in the output. If the absolute condition number is very large, it means that small errors in the input can lead to very large errors in the output, and the function is said to be ill-conditioned. If the absolute condition number is close to 1, it means that small errors in the input lead to small errors in the output, and the function is said to be well-conditioned.

Similarly, the *relative condition number* of $f$ is a measure of the sensitivity of a function's output to perturbations or errors in its input. It is defined as the ratio of the relative change in the output to the relative change in the input. Using the *absolute condition number* in the definition, is is defined as:

$$ \kappa(x) = \lim_{\delta \to 0^{+}} \sup_{\|h\| < \delta} \left( \frac{\|f(x+h) - f(x)\|}{\|h\|} \bigg/ \frac{\|x\|}{\|x\|} \right) = \frac{\|x\|}{\|f(x)\|} \hat{\kappa}(x) $$

The relative condition number tells us how much the output of a function changes relative to changes in the input, and it is often used to assess the stability and accuracy of numerical algorithms. A function with a large condition number is called *ill-conditioned*. Small changes to the input of an ill-conditioned function may produce large changes in output. It is important to know if a function is ill-conditioned because floating point representation almost always introduces some input error, and therefore the outputs of ill-conditioned functions cannot be trusted.

The *condition number* of a matrix $A$, $\kappa(A) = \|A\|\|A^{-1}\|$, is an upper bound on the condition
number for many of the common problems associated with the matrix, such as solving the system
$Ax = b$. If $A$ is square but not invertible, then $\kappa(A) = \infty$ by convention. To compute $\kappa(A)$, we often
use the matrix 2-norm, which is the largest singular value $\sigma_{max}$ of $A$. Recall that if $\sigma$ is a singular
value of $A$, then $\frac{1}{\sigma}$ is a singular value of $A^{-1}$. Thus, we have that $$\kappa(A) = \frac{\sigma_{max}}{\sigma_{min}}$$ which is also a valid equation for non-square matrices.

### Problem 1. 
- Write a function that accepts a matrix A and computes its condition number. Use *scipy.linalg.svd()*, or *scipy.linalg.svdvals()* to compute the singular values of $A$. If the smallest singular value is 0, return $\infty$.

In [4]:
# Problem 1
def matrix_cond(A):
    """Calculate the condition number of A with respect to the 2-norm."""
    # Find the eigenvalues of A
    _, vals, _ = np.linalg.svd(A)

    # Check for 0 eigenvalue being the lowest
    if vals[-1] == 0:
        return np.inf
    else:
        return vals[0]/vals[-1]

## The Wilkinson Polynomial
Let $f : \mathbb{C}^{n+1} \to \mathbb{C}^n$ be the function that maps a collection of $n + 1$ coefficients $(c_n, c_{n−1}, \ldots, c_0)$ to the $n$ roots of the polynomial $c_nx^n + c_{n−1}x^{n−1} + \ldots + c_2x^2 + c_1x + c_0$. Finding polynomial roots is an extremely ill-conditioned problem in general, so the condition number of f is likely very large. To see this, consider the Wilkinson polynomial, made famous by James H. Wilkinson in 1963:

$$ w(x) = \prod_{r=1}^{20} (x - r) = x^{20} - 210x^{19} + 20615x^{18} - 1256850x^{17} + \cdots $$

Let $\tilde{w}(x)$ be $w(x)$ where the coefficient on $x^{19}$ is very slightly perturbed from $−210$ to $−209.9999999$.
The following code computes and compares the roots of $\tilde{w}(x)$ and $w(x)$ using NumPy and SymPy:

In [5]:
def prob2():
    """Randomly perturb the coefficients of the Wilkinson polynomial by
    replacing each coefficient c_i with c_i*r_i, where r_i is drawn from a
    normal distribution centered at 1 with standard deviation 1e-10.
    Plot the roots of 100 such experiments in a single figure, along with the
    roots of the unperturbed polynomial w(x).

    Returns:
        (float) The average absolute condition number.
        (float) The average relative condition number.
    """
    # Get the exact Wilkinson polynomial coefficients using SymPy.
    x, i = sy.symbols('x i')
    w = sy.poly_from_expr(sy.product(x-i, (i, 1, 20)))[0]
    w_coeffs = np.array(w.all_coeffs())

    abs_cond = []
    rel_cond = []
    fig = go.Figure()
    showlegend = False
    for i in range(100):
        if i == 99:
            showlegend = True
        # Perturb the coefficients of the Wilkinson Polynomial slightly
        r = np.random.normal(1, 1e-10, 21)
        c_coeffs = w_coeffs*r

        # Get and sort perturbed roots
        c_roots = np.sort(np.roots(np.poly1d(c_coeffs)))
        new_roots = np.sort(np.roots(np.poly1d(w_coeffs)))

        # Create scatter trace for perturbed roots
        trace = go.Scatter(x=c_roots.real, y=c_roots.imag, 
                           mode='markers', 
                           marker=dict(size=5, opacity=0.4, color='#ff7f0e'),
                           name='Perturbed', showlegend=showlegend)
        fig.add_trace(trace)
        
        # Find relative and absolute condition number
        a = np.linalg.norm((new_roots - c_roots), np.inf)/np.linalg.norm(r, np.inf)
        abs_cond.append(a)
        rel_cond.append(a*np.linalg.norm((w_coeffs), np.inf)/np.linalg.norm(new_roots, np.inf))

    # Calculate the mean of the absolute and relative condition numbers
    mean_abs_cond = np.mean(abs_cond)
    mean_rel_cond = np.mean(rel_cond)
    # Create scatter trace for unperturbed roots
    trace = go.Scatter(x=new_roots.real, y=new_roots.imag, 
                       mode='markers', 
                       marker=dict(size=5, color='slateblue'),
                       name='Unperturbed')
    fig.add_trace(trace)
    
    # Set layout
    fig.update_layout(title='Perturbed Wilkinson Polynomial Coefficients', 
                      showlegend=True, 
                      xaxis_title="x", 
                      yaxis_title="y")
    
    
    fig.add_annotation(dict(font=dict(size=12),
                                        x=-0.07,
                                        y=-0.15,
                                        showarrow=False,
                                        text=f'Average Relative Condition Number: {mean_rel_cond}',
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    fig.add_annotation(dict(font=dict(size=12),
                                        x=-0.07,
                                        y=-0.22,
                                        showarrow=False,
                                        text=f'Average Absolute Condition Number: {mean_abs_cond}',
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    
    # Show plot
    fig.show()

    return mean_abs_cond, mean_rel_cond

In [6]:
prob2()

(4.872246518092125, 3.36279899070096e+18)

## Calculating Eigenvalues
Let $f : M_n(\mathbb{C}) \to \mathbb{C}^n$ be the function that maps an $n × n$ matrix with complex entries to its $n$
eigenvalues. This problem is well-conditioned for symmetric matrices, but it can be extremely ill-conditioned for non-symmetric matrices. Let $A$ be an $n × n$ matrix and let $\lambda$ be the vector of the $n$ eigenvalues of $A$. If $\tilde{A} = A + H$ is a pertubation of $A$ and $\tilde{\lambda}$ are its eigenvalues, then the condition numbers of $f$ can be estimated by the equations:

$$ \hat{\kappa}(A) = \frac{\|\lambda - \tilde{\lambda}\|}{\|H\|} \text{,} \quad \quad \kappa(A) = \frac{\|A\|}{\|\lambda\|}\hat{\kappa}(A) $$

where $\|\cdot\|$ is the matrix 2-norm. The following code computes the condition number of $f$ for a random $n × n$ matrix $A$ and its perturbation $\tilde{A}$.

In [7]:
# Helper function
def reorder_eigvals(orig_eigvals, pert_eigvals):
    """Reorder the perturbed eigenvalues to be as close to the original eigenvalues as possible.
    
    Parameters:
        orig_eigvals ((n,) ndarray) - The eigenvalues of the unperturbed matrix A
        pert_eigvals ((n,) ndarray) - The eigenvalues of the perturbed matrix A+H
        
    Returns:
        ((n,) ndarray) - the reordered eigenvalues of the perturbed matrix
    """
    n = len(pert_eigvals)
    sort_order = np.zeros(n).astype(int)
    dists = np.abs(orig_eigvals - pert_eigvals.reshape(-1,1))
    for _ in range(n):
        index = np.unravel_index(np.argmin(dists), dists.shape)
        sort_order[index[0]] = index[1]
        dists[index[0],:] = np.inf
        dists[:,index[1]] = np.inf
    return pert_eigvals[sort_order]

# Problem 3
def eig_cond(A):
    """Approximate the condition numbers of the eigenvalue problem at A.

    Parameters:
        A ((n,n) ndarray): A square matrix.

    Returns:
        (float) The absolute condition number of the eigenvalue problem at A.
        (float) The relative condition number of the eigenvalue problem at A.
    """
    # construct a matrix with complex entries where the real 
    # and imaginary parts are drawn from normal distributions
    reals = np.random.normal(0, 1e-10, A.shape)
    imags = np.random.normal(0, 1e-10, A.shape)
    H = reals + 1j*imags

    # Compute eigenvalues and reorder
    eigA = la.eigvals(A)
    eigAH = la.eigvals(A+H)
    eigAH = reorder_eigvals(eigA, eigAH)

    # Compute absolute and relative condition numbers
    abs_cond = la.norm((eigA - eigAH), ord=2)/la.norm(H, ord=2)
    rel_cond = abs_cond*la.norm(A, ord=2)/la.norm(eigA, ord=2)

    return abs_cond, rel_cond


### Problem 4. 

Write a function that accepts bounds $[x_{\text{min}}, x_{\text{max}}, y_{\text{min}}, y_{\text{max}}]$ and an integer *res*. Using the function from Problem 3, compute the relative condition number of the eigenvalue problem for the $2 × 2$ matrix
$$\begin{bmatrix}
1 & x\\
y & 1
\end{bmatrix}$$
at every point of an evenly spaced $\text{res} × \text{res}$ grid over the domain $[x_{\text{min}}, x_{\text{max}}] \times [y_{\text{min}}, y_{\text{max}}]$ and plot these estimated relative condition numbers.

In [8]:
# Problem 4
def prob4(domain=[-100, 100, -100, 100], res=50):
    """Create a grid [x_min, x_max] x [y_min, y_max] with the given resolution. For each
    entry (x,y) in the grid, find the relative condition number of the
    eigenvalue problem, using the matrix   [[1, x], [y, 1]]  as the input.
    Use plotly to plot the condition number over the entire grid.

    Parameters:
        domain ([x_min, x_max, y_min, y_max]):
        res (int): number of points along each edge of the grid.
    """
    # Create linspaces
    x = np.linspace(domain[0], domain[1], res)
    y = np.linspace(domain[2], domain[3], res)

    # Init matrix
    vals = np.empty((res,res), dtype=np.float64)

    # Populate matrix
    for i, xi in enumerate(x):
        for j, yi in enumerate(y):
            matrix = np.array([[1, xi], [yi, 1]], dtype=np.float64)
            _, rel = eig_cond(matrix)

            vals[i, j] = rel

    fig = go.Figure(data=go.Heatmap(x=x, y=y, z=vals, colorscale="blackbody"))
    fig.update_layout(title="Estimated Relative Condition Numbers")
    fig['layout'].update(width=650, height=650, autosize=False)
    fig.show()

In [9]:
prob4()

<hr>

## Stability

The stability of an algorithm refers to the property that small changes in the input should lead to only small changes in the output. In other words, a stable algorithm is one that produces consistent results even when the input data is perturbed slightly.

A stable algorithm is desirable because it ensures that the output is reliable and accurate. On the other hand, an unstable algorithm may produce results that are significantly different from the correct values if the input data is even slightly perturbed. Such algorithms are generally considered to be unreliable and may not be suitable for use in many applications.

For numerical algorithms, stability is often closely related to the condition number of the problem being solved. A problem with a high condition number is said to be ill-conditioned, and algorithms that solve such problems may be prone to numerical instability. Therefore, it is important to consider both the stability of the algorithm and the condition number of the problem when choosing a numerical method for a particular application.

Let $f : \R^m \to \R^n$ be a problem to be solved, as in the previous section on conditioning, and let $\tilde{f}$ be an actual algorithm for solving the problem. The forward error of $f$ at $x$ is $\| f(x) − \tilde{f}(x) \| $, and the relative forward error of $f$ at $x$ is $$\frac{\|f(x) - \tilde{f}(x)\|}{\|f(x)\|}$$ An algorithm is called *stable* if its relative forward error is small.

## Least Squares
Ordinary Least Squares (OLS) is a common method used in regression analysis to estimate the parameters of a linear regression model. The goal of OLS is to find the line that best fits the data by minimizing the sum of the squared errors between the observed data points and the predicted values.

In the context of algorithm stability, OLS is considered a stable algorithm because it is well-behaved and has a unique solution. This means that small changes in the input data will only lead to small changes in the output, and the algorithm is not likely to produce widely varying or unpredictable results.

One reason for the stability of OLS is that it is a deterministic algorithm that uses a closed-form solution to calculate the model parameters. This means that the same input data will always produce the same output, which can be helpful for debugging and testing.

Another factor that contributes to the stability of OLS is the fact that it is a linear algorithm. This means that the relationship between the input variables and the output is linear and can be easily modeled using a straight line. Linear models tend to be more stable than non-linear models because they are less susceptible to overfitting and can be more easily interpreted.

Mathematically, the OLS problem is to find the $x$ that  minimizes $\| Ax − b \|_2$ for fixed $A$ and $b$. It can be shown that an equivalent problem is finding the solution of $A^HAx = A^Hb$, called the normal equations. A common application of least squares is polynomial approximation. Given a set of $m$ data points $\{(x_k, y_k)\}_{m}^{k=1}$, the goal is to find the set of coefficients $\{c_i\}_{n}^{i=0}$ such that 

$$y_k \approx c_nx^n_k + c_{n−1}x^{n−1}_k + \ldots + c_2x^2_k + c_1x_k + c_0$$
for all $k$, with the smallest possible error. These $m$ linear equations yield the linear system

$$ Ax = 

\begin{bmatrix}
x^{n}_1 & x^{n-1}_1 & \ldots & x^{2}_1 & x_1 & 1\\
x^{n}_2 & x^{n-1}_2 & \ldots & x^{2}_2 & x_2 & 1\\
x^{n}_3 & x^{n-1}_3 & \ldots & x^{2}_3 & x_3 & 1\\
\vdots & \vdots &            & \vdots & \vdots & \vdots\\
x^{n}_m & x^{n-1}_m & \ldots & x^{2}_m & x_m & 1\\
\end{bmatrix}

\begin{bmatrix}
c_{n}\\
c_{n-1}\\
\vdots\\
c_2\\
c_1\\
c_0\\
\end{bmatrix}

=

\begin{bmatrix}
y_1\\
y_2\\
y_3\\
\vdots\\
y_m\\
\end{bmatrix}

= 
b

$$

In [16]:
def prob5(n, show_plot=False):
    """Approximate the data from "stability_data.npy" on the interval [0,1]
    with a least squares polynomial of degree n. Solve the least squares
    problem using the normal equation and the QR decomposition, then compare
    the two solutions by plotting them together with the data. Return
    the mean squared error of both solutions, ||Ax-b||_2.

    Parameters:
        n (int): The degree of the polynomial to be used in the approximation.
        show_plot (bool): Whether or not to show the plot.

    Returns:
        (float): The forward error using the normal equations.
        (float): The forward error using the QR decomposition.
        (plotly.graph_objects.Figure): A plot of the data and the two approximations.
    """
    # Load data
    xk, yk = np.load("stability_data.npy").T
    
    # Construct Vandermonde matrix
    A = np.vander(xk, n+1)

    # Solve using la.inv() for normal equations.
    x_vals1 = la.inv(A.T @ A) @ A.T @ yk

    # Solve using la.qr
    q, r = la.qr(A, mode='economic')
    x_vals2 = la.solve_triangular(r, q.T@yk)

    # Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xk, y=yk, mode='markers', marker=dict(color='black', size=3), name="Data vals", showlegend=False))
    fig.add_trace(go.Scatter(x=xk, y=np.polyval(x_vals1, xk), mode='lines', name="Least Squares", line=dict(color='red'), showlegend=False))
    fig.add_trace(go.Scatter(x=xk, y=np.polyval(x_vals2, xk), mode='lines', name="QR", line=dict(color='blue'), showlegend=False))
    fig.update_layout(title="Problem 5 Plot", xaxis_title="x", yaxis_title="y", legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    
    if show_plot:
        fig.show()
    
    #Calculate forward error using the normal equations
    forward_normal = la.norm((A@x_vals1 - yk), ord=2)

    #Calculate forward error using the QR decomposition
    forward_qr = la.norm((A@x_vals2 - yk), ord=2)

    return forward_normal, forward_qr, fig

In [22]:
fig = make_subplots(rows=1, cols=4, subplot_titles=("n=4", "n=8", "n=14", "n=21"))
total_error_ls = []
total_error_qr = []

# Plot the approximations for n=4, 8, 14, 21
for i, n in enumerate([4, 8, 14, 21]):
    # Solve the least squares problem using the normal equations and the QR decomposition
    forward_error1, forward_error2, fig_i = prob5(n)
    
    # Add the plots to the subplot
    fig.add_trace(fig_i.data[1], row=1, col=i+1)
    fig.add_trace(fig_i.data[2], row=1, col=i+1)
    fig.add_trace(fig_i.data[0], row=1, col=i+1)
    
    fig.update_yaxes(range=[0, 4], row=1, col=i+1)
    fig.update_xaxes(range=[0, 1], row=1, col=i+1)

    # Store the forward errors
    total_error_ls.append(forward_error1)
    total_error_qr.append(forward_error2)

# Show
fig.update_traces(showlegend=True, col=1, row=1)
fig.update_layout(title_text="Stability of Least Squares Polynomial Approximation using Polynomials of degree n")
fig.show()

# Plot the forward error for n=4, 8, 14, 21
fig = make_subplots(rows=1, cols=4, subplot_titles=("n=4", "n=8", "n=14"))
for err1, err2, i in zip(total_error_ls, total_error_qr, range(4)):
    fig.add_trace(go.Bar(x=["Least Squares", "QR"], y=[err1, err2], name=f"n={n}"), row=1, col=i+1)
fig.update_layout(title="Approximation Error Comparison")
fig.show()

## Catastrophic Cancellation

Catastrophic cancellation is a phenomenon that can occur in numerical computations where the subtraction of two nearly equal numbers produces a result that has lost significant digits of precision. This loss of precision can result in errors that are much larger than the actual accuracy of the computation, hence the term "catastrophic".

When two numbers are subtracted, the relative error of the result is magnified when the numbers being subtracted are very close in value. This can happen, for example, when subtracting two floating-point numbers that differ only in their least significant bits. In this case, the result may be a number that has lost many of its least significant bits, which can significantly affect the accuracy of subsequent calculations that depend on this result.

For example, consider the simple problem of computing  $\sqrt{a} - \sqrt{b}$ The computation can be done directly with subtraction, or by performing the equivalent division 

$$\begin{align*}

\sqrt{a} - \sqrt{b} &= (\sqrt{a} - \sqrt{b}) * \frac{\sqrt{a} + \sqrt{b}}{\sqrt{a} + \sqrt{b}}\\ &= \frac{a-b}{\sqrt{a} + \sqrt{b}}

\end{align*}$$


In [41]:
from math import sqrt # np.sqrt() fails for very large numbers.

a = 10**20 + 1
b = 10**20
print(sqrt(a) - sqrt(b)) # Do the subtraction directly.
if a != b:
    print("Information has been lost!\n")

print((a - b) / (sqrt(a) + sqrt(b))) # Use the alternative formulation.
print('Much better!')


0.0
Information has been lost!

5e-11
Much better!


### Problem 6

Let $I(n) = \int_{0}^{1} x^ne^{x-1}dx$. It can be shown that for a positive integer n, $$I(n) = (-1)^n(!n - \frac{n!}{e})$$ where  $!n = n!\sum_{k=0}^{n} \frac{(-1)^k}{k!}$ Write a function to do the following:

- Use SymPy’s *sy.integrate()* to evaluate the integral form of $I(n)$ for $n = 5, 10, \ldots , 50$. Convert the symbolic results of each integration to a float. Since this is done symbolically, these values can be accepted as the true values of $I(n)$.

- Use the definition above to compute $I(n)$ for the same values of $n$.

- Plot the relative forward error of the results computed in step 2 at each of the given values of $n$.

In [36]:
def prob6():
    """For n = 5, 10, ..., 50, compute the integral I(n) using SymPy (the
    true values) and the subfactorial formula (may or may not be correct).
    Plot the relative forward error of the subfactorial formula for each
    value of n. Use a log scale for the y-axis.
    """
    # Create the list of n
    n = np.arange(5, 51, 5)

    # Init vars
    rel_forward_error = []
    x, N = sy.symbols('x, N')
    integral = x**N * sy.exp(x - 1)
    sub = (-1)**N*(sy.subfactorial(N) - sy.factorial(N)/np.e)

    # For each n
    for i in tqdm(n):
        # Compute integral
        int_val = integral.subs({N:i})
        result = float(sy.integrate(int_val, (x, 0, 1)))

        # Compute the subfactorial way, and compute relative forward error
        sub_res = sub.subs({N:i})
        rel_for = np.abs(sub_res - result)/np.abs(result)
        rel_forward_error.append(float(rel_for))


    # Add the scatter trace to the figure
    fig = go.Figure(go.Scatter(x=n, 
                               y=rel_forward_error, 
                               mode='lines+markers',
                               name='Relative Forward Error'))

    # Update the layout 
    fig.update_layout(title='Problem 6 Plot',
                      xaxis_title='n',
                      yaxis_title='Relative Forward Error',
                      yaxis_type='log')
    
    # Show
    fig.show()

In [37]:
prob6()

  0%|          | 0/10 [00:00<?, ?it/s]